# Blockchain-Distributed-IDS - Client Node  

This notebook runs on a **client node** in the Blockchain-Distributed-IDS system. It trains a local Intrusion Detection System (IDS) model and participates in **Federated Learning** using the Flower framework.  

## Author  
**Charles Stolz**  
cstolz2@und.edu  

## Acknowledgments  
This project builds upon the following open-source contributions:  

### Flower Federated Learning Framework  
- Used for decentralized model training and updates across IDS nodes.  
- **Documentation:** [Flower AI Docs](https://flower.ai/docs/)  
- **GitHub:** [Flower GitHub](https://github.com/adap/flower)  
- **Reference Paper:** Beutel, D.J., Topal, T., Mathur, A. et al. (2020). *Flower: A Friendly Federated Learning Framework.*  


In [110]:
import json
import logging
import numpy as np
import pandas as pd
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [111]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [112]:
#!pip install flwr tensorflow pandas numpy scikit-learn imbalanced-learn pyarrow

In [113]:
CONFIG_PATH = "../config/client_config.json"
with open(CONFIG_PATH, "r") as f:
    config = json.load(f)

In [114]:
CLIENT_ID = config["client_id"]
SERVER_ADDRESS = config["server_address"]
BATCH_SIZE = config["batch_size"]
LEARNING_RATE = config["learning_rate"]
USE_GPU = config["use_gpu"]
DATASET_PATH = config["dataset_path"]

In [115]:
# Disable GPU Raspberry Pi
if not USE_GPU:
    tf.config.set_visible_devices([], "GPU")
    logger.info("using CPU.")

INFO:__main__:using CPU.


In [116]:
df = pd.read_csv(DATASET_PATH)

In [117]:
# extract features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [118]:
logger.info(f"dataset shape: {X.shape}")

INFO:__main__:dataset shape: (100000, 78)


In [119]:
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

In [120]:
#scale feeatures
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [121]:
import collections
# debug before oversampling
original_counts = collections.Counter(y)
print(f"Original class distribution: {original_counts}")

Original class distribution: Counter({'BENIGN': 50000, 'DoS Hulk': 20727, 'PortScan': 14163, 'DDoS': 11489, 'DoS GoldenEye': 956, 'FTP-Patator': 745, 'SSH-Patator': 534, 'DoS Slowhttptest': 511, 'DoS slowloris': 508, 'Bot': 163, 'Web Attack � Brute Force': 138, 'Web Attack � XSS': 60, 'Infiltration': 4, 'Web Attack � Sql Injection': 2})


In [122]:
# convert multi-class labels into binary labels (Attack = 1, Benign = 0)
df["label_binary"] = df.iloc[:, -1].apply(lambda x: 0 if x == "BENIGN" else 1)
y = df["label_binary"].values.astype(np.int32)

In [123]:
# oversampling
ros = RandomOverSampler(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [124]:
oversampled_counts = collections.Counter(y_resampled)
print(f"After oversampling: {oversampled_counts}")

After oversampling: Counter({1: 50000, 0: 50000})


In [125]:
# split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [126]:
def build_model():
    model = Sequential([
        Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(32, activation="relu"),
        Dense(1, activation="sigmoid")  # Binary classification output
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                  loss="binary_crossentropy",
                  metrics=["accuracy", "Precision", "Recall", "AUC"])
    
    return model

In [127]:
logger.info("initializing model")
model = build_model()

INFO:__main__:initializing model
/home/rtikes/Blockchain-Distributed-IDS/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [128]:
class FlowerClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return model.get_weights()

    def set_parameters(self, parameters):
        model.set_weights(parameters)

    def fit(self, parameters, config):
        logger.info("received training request from server start training")
        self.set_parameters(parameters)
        history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=7, verbose=2, validation_data=(X_val, y_val))
        logger.info(f"Training completed. Final Loss: {history.history['loss'][-1]}")
        return self.get_parameters(config), len(X_train), {}

    def evaluate(self, parameters, config):
        logger.info("evaluating model")
        self.set_parameters(parameters)
        loss, accuracy, precision, recall, auc = model.evaluate(X_val, y_val)
        logger.info(f"Evaluation completed. Loss: {loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUC: {auc}")
        return loss, len(X_val), {"accuracy": accuracy, "precision": precision, "recall": recall, "auc": auc}

In [ ]:
# Start Flower client
logger.info(f" starting flower Client {CLIENT_ID}. connecting to server at {SERVER_ADDRESS}...")
fl.client.start_numpy_client(server_address=SERVER_ADDRESS, client=FlowerClient())

INFO:__main__: starting flower Client node-beta. connecting to server at 192.168.0.51:9091...
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use 

Epoch 1/7
1250/1250 - 14s - 12ms/step - AUC: 0.9810 - Precision: 0.9239 - Recall: 0.9392 - accuracy: 0.9309 - loss: 0.1831 - val_AUC: 0.9931 - val_Precision: 0.9463 - val_Recall: 0.9772 - val_accuracy: 0.9609 - val_loss: 0.1025
Epoch 2/7
1250/1250 - 8s - 7ms/step - AUC: 0.9917 - Precision: 0.9492 - Recall: 0.9743 - accuracy: 0.9610 - loss: 0.1102 - val_AUC: 0.9946 - val_Precision: 0.9360 - val_Recall: 0.9933 - val_accuracy: 0.9627 - val_loss: 0.0917
Epoch 3/7
1250/1250 - 8s - 7ms/step - AUC: 0.9931 - Precision: 0.9566 - Recall: 0.9790 - accuracy: 0.9673 - loss: 0.0985 - val_AUC: 0.9959 - val_Precision: 0.9674 - val_Recall: 0.9889 - val_accuracy: 0.9778 - val_loss: 0.0768
Epoch 4/7
1250/1250 - 8s - 7ms/step - AUC: 0.9942 - Precision: 0.9596 - Recall: 0.9817 - accuracy: 0.9702 - loss: 0.0883 - val_AUC: 0.9962 - val_Precision: 0.9704 - val_Recall: 0.9798 - val_accuracy: 0.9750 - val_loss: 0.0736
Epoch 5/7
1250/1250 - 8s - 7ms/step - AUC: 0.9947 - Precision: 0.9615 - Recall: 0.9840 - accur

INFO:__main__:Training completed. Final Loss: 0.0735979825258255
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message 88d34d27-b898-4c8f-846a-ce1ef6e78f0e
INFO:flwr:Received: evaluate message 88d34d27-b898-4c8f-846a-ce1ef6e78f0e
INFO:__main__:evaluating model


625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - AUC: 0.9977 - Precision: 0.9703 - Recall: 0.9894 - accuracy: 0.9795 - loss: 0.0585


INFO:__main__:Evaluation completed. Loss: 0.059482164680957794, Accuracy: 0.9794999957084656, Precision: 0.9695456027984619, Recall: 0.9901000261306763, AUC: 0.9973678588867188
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: train message 78e0e70a-72b5-466d-947e-21db1d4df1bf
INFO:flwr:Received: train message 78e0e70a-72b5-466d-947e-21db1d4df1bf
INFO:__main__:received training request from server start training


...

INFO:__main__:Training completed. Final Loss: 0.04140673950314522
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message baa6a20e-13b4-45f0-be03-045e540af6ac
INFO:flwr:Received: evaluate message baa6a20e-13b4-45f0-be03-045e540af6ac
INFO:__main__:evaluating model


625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - AUC: 0.9995 - Precision: 0.9820 - Recall: 0.9960 - accuracy: 0.9888 - loss: 0.0304


INFO:__main__:Evaluation completed. Loss: 0.030611032620072365, Accuracy: 0.989300012588501, Precision: 0.9821639657020569, Recall: 0.9966999888420105, AUC: 0.9994180202484131
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message 61720e3b-499b-4c64-89bc-11cc2f479301
INFO:flwr:Received: evaluate message 61720e3b-499b-4c64-89bc-11cc2f479301
INFO:__main__:evaluating model


625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - AUC: 0.9996 - Precision: 0.9840 - Recall: 0.9958 - accuracy: 0.9898 - loss: 0.0259


INFO:__main__:Evaluation completed. Loss: 0.02630488947033882, Accuracy: 0.9904500246047974, Precision: 0.9845864772796631, Recall: 0.9965000152587891, AUC: 0.9994423389434814
INFO :      Sent reply
INFO:flwr:Sent reply
